In [1]:
# Connect to workspace
from azureml.core import Workspace
ws = Workspace.from_config()

In [9]:
# Create Docker environment
from azureml.core import Environment

myenv = Environment(name="myenv")
myenv.docker.enabled = True
with open("Dockerfile", "r") as f:
    dockerfile=f.read()
myenv.docker.base_dockerfile = dockerfile
myenv.python.user_managed_dependencies = True

Property base_image is mutually exclusive with base_dockerfile. Reset base_image to None


In [11]:
# Configuration
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(source_directory = "scripts",
                                              entry_script="entry_script.py",
                                              environment=myenv)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


In [9]:
# Deploy a k8s
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'inference-k8s'
compute_config = AksCompute.provisioning_configuration(
    cluster_purpose='DevTest',
    vm_size='Standard_D11_v2',
    agent_count=2,
    location='westeurope')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Creating...........................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"


In [10]:
# Configure service
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

In [12]:
# Deploy!!!
from azureml.core.model import Model

model = ws.models['diabetes_model']
service = Model.deploy(workspace=ws,
                       name = 'diabetes-serv',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................................................................................................................................
Succeeded
AKS service creation operation finished, operation "Succeeded"


In [15]:
import requests
import json

# Auth
primary_key, secondary_key = service.get_keys()

# An array of new data cases
x_new = [[6,148,72,35,0,33.6,0.627,50],
         [11,138,76,0,0,33.2,0.42,35]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + primary_key}

# Call the service
response = requests.post(url = 'http://23.100.12.190:80/api/v1/service/diabetes-serv/score',
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
print(response.json())

[1, 1]
